In [ ]:
import arcpy
import arcgis
import pandas as pd
import numpy as np
import datetime
import requests
import matplotlib.pyplot as plt
import os

In [ ]:
# Define local directory
local_directory = r"E:\coursework\CSCI 5527\CSCI 5527\data"
# Create the local directory if it doesn't exist
if not os.path.exists(local_directory):
    os.makedirs(local_directory)

arcpy.env.workspace = r"E:\coursework\CSCI 5527\CSCI 5527\data"

sr = arcpy.SpatialReference(4326)

In [ ]:
#url
# MN temperature data in 2023
temperature_url = r"https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&year=2021&month=7"

In [ ]:
#request and pull data via url
temperature_response = requests.get(temperature_url)

temperature_json = temperature_response.json()["features"]

#define a fuction to extract data
def ExtractData(data,col,jasonfield):
    data[col] = data[jasonfield].apply(lambda x: dict(x)[col])

#transform geojason as dataframe
temperature_rawdf = pd.DataFrame.from_records(temperature_json)

#property info
temperature_col = ["id", "date", "name", "max_tmpf", "min_tmpf","precip"]

for col_name in temperature_col:
    ExtractData(temperature_rawdf,col_name,"properties")

#location info
temperature_rawdf['x'] = temperature_rawdf["geometry"].apply(lambda x: dict(x)["coordinates"][0])
temperature_rawdf['y'] = temperature_rawdf["geometry"].apply(lambda x: dict(x)["coordinates"][1])

temperature_df = temperature_rawdf[["id", "date", "name", "max_tmpf", "min_tmpf",'precip','x','y']].copy()

#missing values (temperature and location)
temperature_df = temperature_df.dropna(subset=["max_tmpf", "min_tmpf","precip","x", "y"])
temperature_df["date"] = temperature_df["date"].astype('datetime64[ns]')
temperature_df["day"] = pd.DatetimeIndex(temperature_df["date"]).day

temperature_df

,id,date,name,max_tmpf,min_tmpf,precip,x,y,day
0,MN001,2021-07-01,Twin Lakes I-35 Mile Post 1,85.099990,58.09999,0.0,-93.354057,43.508331,1
1,MN002,2021-07-01,Silver Lake TH 7 Mile Post 1,85.460014,59.89999,0.0,-94.119100,44.906800,1
2,MN003,2021-07-01,Little Chicago I-35 Mile Post 70,84.919975,57.74002,0.0,-93.292427,44.478500,1
3,MN004,2021-07-01,Rush City I-35 Mile Post 157,80.240020,56.66001,0.0,-92.992752,45.642921,1
4,MN005,2021-07-01,Rutledge I-35 Mile Post 198,80.599990,57.38000,0.0,-92.838562,46.212570,1
...,...,...,...,...,...,...,...,...,...
3032,MN093,2021-07-31,Silver Cliff MN-61 Mile Post 30,78.980000,70.51998,0.0,-91.591553,47.069969,31
3034,MN095,2021-07-31,Blatnick Bridge South Abutment,79.699990,73.21998,0.0,-92.097100,46.734100,31
3035,MN096,2021-07-31,Albert Lea I-35 Mile Post 30,78.980000,56.66001,0.0,-93.277618,43.920101,31
3036,MN097,2021-07-31,Atkinson Bridge I-35 Mile Post 231,77.180000,58.28000,0.0,-92.531900,46.627800,31


In [ ]:
#bounding box
temperature_df =temperature_df[(temperature_df["x"] > -94.16) & (temperature_df["x"] < -92.66) &
                              (temperature_df["y"] > 44.28) & (temperature_df["y"] < 45.67)]
temperature_df

,id,date,name,max_tmpf,min_tmpf,precip,x,y,day
1,MN002,2021-07-01,Silver Lake TH 7 Mile Post 1,85.460014,59.899990,0.0,-94.119100,44.906800,1
2,MN003,2021-07-01,Little Chicago I-35 Mile Post 70,84.919975,57.740020,0.0,-93.292427,44.478500,1
3,MN004,2021-07-01,Rush City I-35 Mile Post 157,80.240020,56.660010,0.0,-92.992752,45.642921,1
16,MN017,2021-07-01,Clearwater I-94 Mile Post 180,86.899990,63.319977,0.0,-94.025612,45.394691,1
21,MN022,2021-07-01,Lester Prairie MN-7 Mile Post 161,85.460014,58.999990,0.0,-93.986400,44.906100,1
...,...,...,...,...,...,...,...,...,...
3025,MN086,2021-07-31,Burnsville I-35W Mile Post 4,85.460014,58.640022,0.0,-93.289558,44.797649,31
3026,MN087,2021-07-31,Maple Grove I-94 Mile Post 217,84.919975,60.619980,0.0,-93.447533,45.092091,31
3027,MN088,2021-07-31,Little Canada I-694 Mile Post 46,86.540020,60.080000,0.0,-93.092361,45.037659,31
3028,MN089,2021-07-31,Cayuga Street Br. I-35E Mile Post 109,85.819980,61.340023,0.0,-93.088127,44.967861,31


In [ ]:
#create weather shapefile
for i in range(31):# create a new feature class
    arcpy.CreateFeatureclass_management(arcpy.env.workspace, f"temperature_{i+1}", "POINT",spatial_reference=sr)

    # add fields to the feature class
    arcpy.AddField_management(f"temperature_{i+1}", "station_id", "Text")
    arcpy.AddField_management(f"temperature_{i+1}", "name", "Text")
    arcpy.AddField_management(f"temperature_{i+1}", "max_tmpf", "Double")
    arcpy.AddField_management(f"temperature_{i+1}", "min_tmpf", 'Double')
    arcpy.AddField_management(f"temperature_{i+1}", "precip", 'Double')

    rows = temperature_df[temperature_df['day']==i+1]

    # insert data into the feature class
    cursor = arcpy.da.InsertCursor(f"temperature_{i+1}", ["SHAPE@", "station_id", "name","max_tmpf",'min_tmpf',"precip"])
    for index,row in rows.iterrows():
        point = arcpy.Point(row[6], row[7])
        cursor.insertRow([point, row[0], row[2],row[3], row[4], row[5]])
    del cursor

In [ ]:
#spatial interpolation
for i in range(31):
    arcpy.ga.IDW(in_features = f"temperature_{i+1}.shp", z_field = 'min_tmpf', out_raster = f"temp_min_{i+1}.tif", cell_size = 0.001)
    arcpy.ga.IDW(in_features = f"temperature_{i+1}.shp", z_field = 'max_tmpf', out_raster = f"temp_max_{i+1}.tif", cell_size = 0.001)
    arcpy.ga.IDW(in_features = f"temperature_{i+1}.shp", z_field = 'precip', out_raster = f"precip_{i+1}.tif", cell_size = 0.001)

In [ ]:
#get grid shapefile
arcpy.JSONToFeatures_conversion('fishnet_3km.json', 'fishnet_3km.shp')
arcpy.management.Project('fishnet_3km.shp','grid.shp',sr)

#grid centriod
arcpy.CreateFeatureclass_management(arcpy.env.workspace, "grid_pnt", "POINT",spatial_reference=sr)

# add fields to the feature class
arcpy.AddField_management("grid_pnt", "x", "Double")
arcpy.AddField_management("grid_pnt", "y", "Double")

# Create an insert cursor to add features to the point shapefile
with arcpy.da.InsertCursor("grid_pnt", ["SHAPE@", "x","y"]) as insert_cursor:
    # Create a search cursor to iterate through the polygons and calculate centroids
    with arcpy.da.SearchCursor("grid", ["SHAPE@TRUECENTROID"]) as search_cursor:
        for row in search_cursor:
            #point = arcpy.Point(row[0])
            (x,y) = row[0]
            # Insert a new point feature with centroid coordinates into the point shapefile
            insert_cursor.insertRow([row[0], x, y])

In [ ]:
#get weather info for each census tract
for i in range(31):
    arcpy.sa.ExtractValuesToPoints('grid_pnt', f"temp_min_{i+1}.tif", f"temperature_min_{i+1}.shp", "INTERPOLATE", "VALUE_ONLY")
    arcpy.sa.ExtractValuesToPoints('grid_pnt', f"temp_max_{i+1}.tif", f"temperature_max_{i+1}.shp", "INTERPOLATE", "VALUE_ONLY")
    arcpy.sa.ExtractValuesToPoints('grid_pnt', f"precip_{i+1}.tif", f"precipitation_{i+1}.shp", "INTERPOLATE", "VALUE_ONLY")
    arcpy.TableToTable_conversion(f"temperature_min_{i+1}.shp", arcpy.env.workspace, f"temperature_min_{i+1}.csv")
    arcpy.TableToTable_conversion(f"temperature_max_{i+1}.shp", arcpy.env.workspace, f"temperature_max_{i+1}.csv")
    arcpy.TableToTable_conversion(f"precipitation_{i+1}.shp", arcpy.env.workspace, f"precipitation_{i+1}.csv")

In [ ]:
dfs_temperature_min = pd.DataFrame()
dfs_temperature_max = pd.DataFrame()
dfs_precipitation = pd.DataFrame()

for i in range(31):
    df = pd.read_csv(os.path.join(local_directory,f'temperature_min_{i+1}.csv'))
    df['day'] = i+1
    dfs_temperature_min = dfs_temperature_min.append(df)

    df = pd.read_csv(os.path.join(local_directory,f'temperature_max_{i+1}.csv'))
    df['day'] = i+1
    dfs_temperature_max = dfs_temperature_max.append(df)

    df = pd.read_csv(os.path.join(local_directory,f'precipitation_{i+1}.csv'))
    df['day'] = i+1
    dfs_precipitation = dfs_precipitation.append(df)

In [ ]:
dfs.head()

,OID_,Id,x,y,RASTERVALU,day
0,0,0,-93.270797,44.484414,57.8473,1
1,1,0,-93.233070,44.484497,58.4259,1
2,2,0,-93.195344,44.484568,59.2519,1
3,3,0,-93.157617,44.484627,60.1624,1
4,4,0,-93.119890,44.484672,60.7997,1


In [ ]:
dfs_weather = pd.merge(dfs_temperature_min, dfs_temperature_max, on = ['x','y','day'], suffixes=('_min', '_max'))
dfs_weather = pd.merge(dfs_weather,dfs_precipitation, on = ['x','y','day'])
dfs_weather = dfs_weather[['x','y','day','RASTERVALU_min','RASTERVALU_max','RASTERVALU']]
dfs_weather = dfs_weather.rename(columns={"RASTERVALU_min": "temp_min", "RASTERVALU_max": "temp_max", "RASTERVALU": "precip"})
dfs_weather.to_csv(os.path.join(local_directory,'weather.csv'))
dfs_weather

,x,y,day,temp_min,temp_max,precip
0,-93.270797,44.484414,1,57.8473,84.9613,0.0
1,-93.233070,44.484497,1,58.4259,85.1599,0.0
2,-93.195344,44.484568,1,59.2519,85.3889,0.0
3,-93.157617,44.484627,1,60.1624,85.4956,0.0
4,-93.119890,44.484672,1,60.7997,85.4918,0.0
...,...,...,...,...,...,...
28670,-93.160150,45.402829,31,59.6970,84.6051,0.0
28671,-93.121816,45.402876,31,59.7210,84.5298,0.0
28672,-93.083483,45.402911,31,59.7095,84.4155,0.0
28673,-93.045150,45.402932,31,59.7147,84.3404,0.0


### Tract

In [ ]:
#tract centriod
arcpy.CreateFeatureclass_management(arcpy.env.workspace, "tract_pnt", "POINT",spatial_reference=sr)

# add fields to the feature class
arcpy.AddField_management("tract_pnt", "x", "Double")
arcpy.AddField_management("tract_pnt", "y", "Double")
arcpy.AddField_management("tract_pnt", "TRACTCE20", "Text")
arcpy.AddField_management("tract_pnt", "GEOID20", "Text")

# Create an insert cursor to add features to the point shapefile
with arcpy.da.InsertCursor("tract_pnt", ["SHAPE@", "x","y","TRACTCE20", "GEOID20"]) as insert_cursor:
    # Create a search cursor to iterate through the polygons and calculate centroids
    with arcpy.da.SearchCursor("tract", ["SHAPE@TRUECENTROID","TRACTCE20","GEOID20"]) as search_cursor:
        for row in search_cursor:
            #point = arcpy.Point(row[0])
            (x,y) = row[0]
            # Insert a new point feature with centroid coordinates into the point shapefile
            insert_cursor.insertRow([row[0], x, y,row[1], row[2]])

In [ ]:
#get weather info for each census tract
for i in range(31):
    arcpy.sa.ExtractValuesToPoints('tract_pnt', f"temp_min_{i+1}.tif", f"temperature_min_{i+1}.shp", "INTERPOLATE", "VALUE_ONLY")
    arcpy.sa.ExtractValuesToPoints('tract_pnt', f"temp_max_{i+1}.tif", f"temperature_max_{i+1}.shp", "INTERPOLATE", "VALUE_ONLY")
    arcpy.sa.ExtractValuesToPoints('tract_pnt', f"precip_{i+1}.tif", f"precipitation_{i+1}.shp", "INTERPOLATE", "VALUE_ONLY")
    arcpy.TableToTable_conversion(f"temperature_min_{i+1}.shp", arcpy.env.workspace, f"temperature_min_{i+1}.csv")
    arcpy.TableToTable_conversion(f"temperature_max_{i+1}.shp", arcpy.env.workspace, f"temperature_max_{i+1}.csv")
    arcpy.TableToTable_conversion(f"precipitation_{i+1}.shp", arcpy.env.workspace, f"precipitation_{i+1}.csv")